# Shadow Detection and Removal

## Introduction

A shadow appears on an area when the light from a source cannot reach the area due to obstruction by an object. 

Shadow in general are of two types (by looking to the videos frames we find that during our project we will meet both of them): hard and soft shadows. The soft shadows retain the texture of the background surface, whereas the hard shadows are too dark and have little texture.

Most of the shadow detection methods need multiple images for camera calibration, But the best technique must be able to extract shadows from a single image where we will consider this technique for our project. 

A shadow detection method is selected based on the mean value of RGB image in A and B planes of LAB equivalent of the image.

<img align="left" src="img/lab_color_space.jpg" style=" width:300px; padding-right: 30px;  " /> 

<u>**The LAB colour space:**</u> The LAB colour space has three channels − L is the Lightness channel, A and B are the two colour channels.

The L channel has values ranging from 0 up to 100, which correspond to different shades from black to white. The A channel has values ranging from −128 up to +127 and gives the red to green ratio. The B channel also has values ranging from −128 up to +127 and gives the yellow to blue ratio. 

Thus, a high value in A or B channel represents a colour having more red or yellow and a low value represents a colour having more green or blue.



## Background

...

...

...

## Implementation

We will apply this task by Detecting the shadows of objects first, then try to remove these shadows.

<u>**1.Shadow Detection:**</u> 

An approach to detect the shadows areas in a single RGB image is convert from RGB to LAB color space. Since the shadow regions are darker and less illuminated than the surroundings, it is easy to locate them in the L channel since the L channel gives lightness information. The B channel values are also lesser in the shadow areas in most of the outdoor images. 

Thus combining the values from L and B channels, the pixels with values less than a threshold are identified as shadow pixels, and others as non-shadow pixels. The method works well only for images whose yellow to blue ratio is maintained within a range.

The mean value of the image in A and B channels are calculated.

The major steps involved in the shadow detection phase are:  
1. Convert the RGB image to a LAB image. 
2. Compute the mean values of the pixels in L, A and B planes of the image separately. 
3. If mean (A) + mean (B) ≤ 256 then Classify the pixels with a value in L ≤(mean(L) – standard deviation (L)/3) as shadow pixels and others as non-shadow pixels. 
4. Else classify the pixels with lower values in both L and B planes as shadow pixels and others as non-shadow pixels.

The shadow detection using this pixel-based method may classify some non shadow pixels as shadow pixels. Isolated pixels are removed using morphological 
operation called cleaning. 

The misclassified pixels are removed using dilation followed by erosion. Also area-based thresholding is done, so that only regions with a number of pixels greater than a threshold can be considered as shadow regions. All these morphological operations will help to eliminate misclassification of pixels.

<u>**2.Shadow Removal and Edge Correction:**</u> 

Shadow removal is done by multiplying R, G and B channels of the shadow pixels using appropriate constants. Each shadow region is considered separately. The ratio of the average of each channel in the near non-shadow region to that in the shadow region is taken as a constant for each channel. The shadow regions achieve almost the same illumination as the non-shadow regions. But over-illumination may occur towards the edges of shadow. 

Since shadow regions are not uniformly illuminated, the same constant for the entire shadow region will create over-illuminated areas near the shadow edges. This is overcome by applying a median filter on the over-illuminated areas. Thus a shadow-free image without over-illuminated edges is obtained.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

from expt_utils import *

In [3]:
img = cv.imread(f'{IMG_DIR}/simple_object_2.jpg')
img_gray = cv.cvtColor(img, cv.COLOR_BGR2RGB)

cv.imshow('Image', img)
cv.imshow('Image Gray', img_gray)

destroy_when_esc()

In [ ]:
def shadow_detection(image):

    lab_image = cv.cvtColor(image, cv.COLOR_RGB2LAB)

    mean_l = np.mean(lab_image[:, :, 0])
    mean_a = np.mean(lab_image[:, :, 1])
    mean_b = np.mean(lab_image[:, :, 2])
    std_l = np.std(lab_image[:, :, 0])

    if mean_a + mean_b <= 256:
        shadow_mask = lab_image[:, :, 0] <= (mean_l - std_l/3)
    else:
        shadow_mask = (lab_image[:, :, 0] < mean_l) & (lab_image[:, :, 2] < mean_b)

    kernel = np.ones((3, 3), np.uint8)
    cleaned_mask = cv.morphologyEx(shadow_mask.astype(np.uint8), cv.MORPH_OPEN, kernel)

    shadow_mask_cleaned = cv.dilate(cleaned_mask, kernel, iterations=1)
    shadow_mask_cleaned = cv.erode(shadow_mask_cleaned, kernel, iterations=1)
    
    # non_shadow_mask_cleaned = ~shadow_mask_cleaned

    result_image = np.copy(image)
    result_image[shadow_mask_cleaned] = [0,0,0]  

    return result_image, shadow_mask_cleaned

In [ ]:
def shadow_remove(image, shadow_mask):
    image_float = image.astype(np.float32)

    shadow_pixels = np.where(shadow_mask)

    avg_rgb_inside_shadow = np.zeros((3,))
    avg_rgb_outside_shadow = np.zeros((3,))

    for channel in range(3):
        avg_rgb_inside_shadow[channel] = np.mean(image_float[shadow_pixels[0], shadow_pixels[1], channel])

    outside_shadow_mask = ~shadow_mask
    outside_shadow_pixels = np.where(outside_shadow_mask)
    
    for channel in range(3):
        avg_rgb_outside_shadow[channel] = np.mean(image_float[outside_shadow_pixels[0], outside_shadow_pixels[1], channel])

    constants = avg_rgb_outside_shadow / avg_rgb_inside_shadow

    for channel in range(3):
        image_float[shadow_pixels[0], shadow_pixels[1], channel] *= constants[channel]

    result_image = np.clip(image_float, 0, 255).astype(np.uint8)

    return result_image

In [ ]:
result_shadow_img, shadow_mask = shadow_detection(img)
final_result_img = shadow_remove(result_shadow_img, shadow_mask)

cv.imshow('Image', img)
cv.imshow('Result Shadow Img', result_shadow_img)
cv.imshow('Shadow Mask', shadow_mask)
cv.imshow('Final Result Img', final_result_img)

destroy_when_esc()

## Conclusion

We found that shadow edge correction is done to reduce the errors in the shadow boundary very well.

We only could reduce the errors in the shadow and not to remove it at all because several factors, for examples:
- Extract shadows only from a single image: this lead the algorithm to Limited Perspective, Ambiguity in Shadow Detection and Loss of Depth Information.
- Non-uniform illumination: The illumination is not uniform in the shadow region. Towards the shadow boundary, diffusion takes place.